Import necessary packages

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

Create a CNN Model

In [2]:
class CNN_model(nn.Module):
  '''
  CNN Model
  Input: Image 28x28x1
  Output: Logit scores for each class
  Architecture: Input --> Conv2d(3x3) --> MaxPool2D(2x2) --> Conv2d(3x3) --> MaxPool2D(2x2) --> FC(64) --FC(10)
  '''
  def __init__(self,input_channels,num_classes):
    super(CNN_model,self).__init__()
    self.device=device
    self.conv1=nn.Conv2d(in_channels=input_channels,out_channels=4,kernel_size=(3,3),stride=(1,1),padding=(1,1)) # 28 x28
    self.pool=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2)) # 14 x 14
    self.conv2=nn.Conv2d(in_channels=4,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1)) # 14 x 14
    self.fc1=nn.Linear(in_features=7*7*8,out_features=32)
    self.fc2=nn.Linear(in_features=32,out_features=num_classes)

  def forward(self,x):
    x=F.relu(self.conv1(x)) # 28 x 28 x 4
    x=self.pool(x)          # 14 x 14 x 4
    x=F.relu(self.conv2(x)) # 14 x 14 x 8
    x=self.pool(x)          # 7 x 7 x 8
    x=x.reshape(x.shape[0],-1)
    x=F.relu(self.fc1(x))   # 32 x 1
    x=self.fc2(x)           # 10 x 1
    return x



Hyperparameters of the CNN model

In [3]:
input_channels=1
num_classes=10
batch_size=64
learning_rate=0.001
num_epochs=4

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

Load the data and create dataloaders

In [5]:
train_dataset=datasets.MNIST(root='./dataset/',train=True,download=True,transform=transforms.ToTensor())
train_dataloader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset=datasets.MNIST(root='./dataset/',train=False,download=True,transform=transforms.ToTensor())
test_dataloader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

Initialize a model and transfer it to GPU 

In [6]:
model=CNN_model(input_channels,num_classes).to(device=device)

Define a loss and optimizer

In [7]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

Create a Training Loop

In [8]:
# Training loop
for epoch in range(num_epochs):
  for batch_idx,(inputs,targets) in enumerate(train_dataloader):
    inputs=inputs.to(device=device)
    targets=targets.to(device=device)
    scores= model(inputs)
    
    loss=criterion(scores,targets)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    


Accuracy Metric

In [9]:
def calculate_accuracy(loader,model):
  if loader.dataset.train:
    print("Training accuracy")
  else:
    print("Test accuracy")
  model.eval()
  num_correct=0
  num_samples=0

  with torch.no_grad():
    for x,y in loader:
      x=x.to(device=device)
      y=y.to(device=device)
      
      score=model(x)
      _,preds=score.max(axis=1)
      num_correct+=(preds==y).sum()
      num_samples+=y.shape[0]
  print(f"Got {num_correct}/{num_samples} with accuracy {(float(num_correct)/float(num_samples))*100} % ")
  model.train()




Compute the Accuracy of Training and Test data

In [10]:
calculate_accuracy(train_dataloader,model)
calculate_accuracy(test_dataloader,model)

Training accuracy
Got 58374/60000 with accuracy 97.28999999999999 % 
Test accuracy
Got 9725/10000 with accuracy 97.25 % 
